Reference:

- Data source:
    <https://data.worldbank.org/topic/climate-change?end=2018&start=1973>

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.1f' % x)

In [2]:
climate_1960_2018 = pd.read_csv('datasets/climate_1960_2018.csv')

In [3]:
climate_1960_2018.drop(columns='Unnamed: 63', inplace=True)

In [4]:
climate_1960_2018.shape

(20064, 63)

In [5]:
indicators = ['Population, total',
              'Population growth (annual %)',
              # greenhouse gas
              'CO2 emissions (kt)',
              'CO2 emissions (metric tons per capita)',
              'Methane emissions (kt of CO2 equivalent)',
              'Nitrous oxide emissions (thousand metric tons of CO2 equivalent)',
              'SF6 gas emissions (thousand metric tons of CO2 equivalent)',
              'PFC gas emissions (thousand metric tons of CO2 equivalent)',
              'HFC gas emissions (thousand metric tons of CO2 equivalent)',
              'Total greenhouse gas emissions (kt of CO2 equivalent)',
              
              'Electric power consumption (kWh per capita)',
              'Energy use (kg of oil equivalent per capita)',
              'Forest area (% of land area)',
              'Forest area (sq. km)',
              'Agricultural land (% of land area)']

climate_filtered_1960_2018 = climate_1960_2018[climate_1960_2018['Indicator Name'].isin(indicators)]
climate_filtered_1960_2018.fillna(0, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [6]:
climate_filtered_1960_2018.drop(columns=['Country Code', 'Indicator Code'], inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [7]:
def df_4each_indicator(glb_df, indicator, world=True):
    df = glb_df[glb_df['Indicator Name'] == indicator]
    df.index = df['Country Name']
    df.drop(columns=['Country Name', 'Indicator Name'], inplace=True)
    df = df.T
    if world:
        df['World'] = df.sum(axis='columns')

    return df

In [8]:
pop_tot_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                       'Population, total')
pop_growthPct_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                             'Population growth (annual %)',
                                             world=False)

co2_kt_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                      'CO2 emissions (kt)')
co2_tons_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                        'CO2 emissions (metric tons per capita)')
methane_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                       'Methane emissions (kt of CO2 equivalent)')
n2o_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                        'Nitrous oxide emissions (thousand metric tons of CO2 equivalent)')
sf6_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                        'SF6 gas emissions (thousand metric tons of CO2 equivalent)')
pfc_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                        'PFC gas emissions (thousand metric tons of CO2 equivalent)')
hfc_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                        'HFC gas emissions (thousand metric tons of CO2 equivalent)')
greenhouse_gas_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                              'Total greenhouse gas emissions (kt of CO2 equivalent)')

elec_power_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                          'Electric power consumption (kWh per capita)')
energy_use_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                          'Energy use (kg of oil equivalent per capita)')
forest_areaPct_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                              'Forest area (% of land area)',
                                              world=False)
forest_areaSQ_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                             'Forest area (sq. km)')
agric_landPct_1960_2018 = df_4each_indicator(climate_filtered_1960_2018,
                                             'Agricultural land (% of land area)',
                                             world=False)

TODO:
- 1 line plot for pop_tot_1960_2018 & pop_growthPct_1960_2018
- 1 line plot for greenhouse_gas_1960_2018, methane_1960_2018, co2_kt_1960_2018
- 1 line plot for co2_tons_1960_2018, n2o_1960_2018, sf6_1960_2018, pfc_1960_2018, hfc_1960_2018
- 1 line plot for elec_power_1960_2018
- 1 line plot for energy_use_1960_2018
- 1 line plot for forest_areaPct_1960_2018 & forest_areaSQ_1960_2018
- 1 line plot for agric_landPct_1960_2018